In [1]:
import tensorflow as tf
import pandas as pd # Pandas DataFrame
import numpy as np # Numpy array manipulation
from sklearn.decomposition import PCA # Princible Component Analysis
from sklearn.preprocessing import Imputer # Missing value imputation

In [2]:
# Load data from CSV
training_data = pd.read_csv('train_signal.csv')

In [3]:
# Peek at the data
training_data.head()

,ID,Type,X0,X1,X2,X3,X4,X5,X6,X7,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,B00000,N,-0.107,-0.100,-0.086,-0.078,-0.071,-0.057,-0.049,-0.035,...,-1.108,-1.072,-1.028,-0.978,-0.912,-0.862,-0.804,-0.724,-0.630,-0.499
1,B00001,N,2.762,3.313,3.863,4.292,4.594,4.623,4.408,3.817,...,-0.107,0.003,0.148,0.241,0.310,0.345,0.368,0.397,0.426,0.438
2,B00002,N,-0.246,-0.200,-0.159,-0.125,-0.101,-0.090,-0.084,-0.078,...,-0.113,-0.038,0.032,0.107,0.165,0.194,0.194,0.159,0.119,0.072
3,B00003,~,0.519,0.778,1.073,1.392,1.672,1.895,2.012,2.023,...,0.037,-0.052,-0.084,-0.099,-0.101,-0.090,-0.067,0.003,0.096,0.179
4,B00004,~,0.011,-0.103,-0.265,-0.371,-0.409,-0.422,-0.418,-0.411,...,0.776,0.829,0.763,0.481,0.126,-0.144,-0.224,-0.250,-0.222,-0.207


In [38]:
# Split into labels (y) and input (X)

# Data from the 3rd feature column onwards are input
X = training_data.values[:,2:]
# Classes/Labels are the type of AF
y = training_data["Type"].values

In [39]:
#print(f"X shape: {X.shape}, y shape: {y.shape}")
print("X shape:  " + str(X.shape))
print("y shape:  " + str(y.shape))

X shape:  (13062, 6000)
y shape:  (13062,)


In [40]:
from keras.utils import to_categorical # Multi-class labels converted to one-hot encoded categories
from sklearn.preprocessing import LabelEncoder # Enumeration for labels

In [41]:
# Convert class labels to integers to then one-hot encode

# Encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

# One-hot encode
print(y.shape)
y = to_categorical(encoded_Y)
print(y.shape)

(13062,)
(13062, 4)


In [42]:
# Deal with missing data

# Replace missing values with Nan
X[X == ''] = np.nan

# np.any(np.isnan(X))

# Replace Nan with median
imputer = Imputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)

In [43]:
print(X)

[[-0.107 -0.1   -0.086 ... -0.724 -0.63  -0.499]
 [ 2.762  3.313  3.863 ...  0.397  0.426  0.438]
 [-0.246 -0.2   -0.159 ...  0.159  0.119  0.072]
 ...
 [-0.364 -0.395 -0.434 ... -0.834 -0.632 -0.472]
 [-0.285 -0.475 -0.596 ...  0.77   0.764  0.754]
 [ 0.975  0.935  0.87  ... -0.109 -0.114 -0.119]]


In [44]:


# Reduce dimensions of data to make neural networks easier to train, prevent overfitting & remove noise

# pca = PCA(n_components=60) # reduce to 60 dimensions
# pca.fit(X)

# print(X.shape)
# X_pca = pca.transform(X)
# print(X_pca.shape) 

In [45]:
from keras.preprocessing import sequence # Keras input preprocessing
from keras.models import Sequential # Keras model types
from keras.layers import Dense, Dropout, Activation # Keras model layers
from keras.layers import Embedding, LSTM
from keras.layers import Input, Dense, concatenate, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers import BatchNormalization, GlobalAveragePooling1D, Permute, Dropout
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD, Adam
from keras.models import Model

In [46]:
def FCN():

    model = Sequential()
    model.add(Conv1D(128, 8, padding = "same", kernel_initializer = "he_uniform"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv1D(128, 8, padding = "same", kernel_initializer = "he_uniform"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(rate=0.25))
    model.add(Conv1D(128, 8, padding = "same", kernel_initializer = "he_uniform"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(rate=0.25))
    model.add(GlobalAveragePooling1D())

    model.add(Dense(100, activation='relu'))
    model.add(Dense(4, activation = "softmax"))

    model.summary
              
    return model

In [47]:

model = FCN()

In [48]:
# Tell model what loss function & optimiser to use
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.01),
              metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [49]:
X_3d = X.reshape((13062, 20, 300))
print(X_3d.shape)
 
model.fit(X_3d, y, epochs=150, batch_size=32, validation_split=1/6)

(13062, 20, 300)
Train on 10885 samples, validate on 2177 samples
Epoch 1/150
10885/10885 [==============================] - 12s 1ms/step - loss: 0.9514 - accuracy: 0.6047 - auc_3: 0.8250 - precision_3: 0.6128 - recall_3: 0.4656 - val_loss: 0.9693 - val_accuracy: 0.6201 - val_auc_3: 0.8390 - val_precision_3: 0.6384 - val_recall_3: 0.5080
Epoch 2/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.8756 - accuracy: 0.6456 - auc_3: 0.8470 - precision_3: 0.6513 - recall_3: 0.5285 - val_loss: 0.9187 - val_accuracy: 0.6284 - val_auc_3: 0.8512 - val_precision_3: 0.6586 - val_recall_3: 0.5371
Epoch 3/150
10885/10885 [==============================] - 11s 995us/step - loss: 0.8212 - accuracy: 0.6706 - auc_3: 0.8571 - precision_3: 0.6680 - recall_3: 0.5497 - val_loss: 0.9051 - val_accuracy: 0.6353 - val_auc_3: 0.8603 - val_precision_3: 0.6734 - val_recall_3: 0.5561
Epoch 4/150
10885/10885 [==============================] - 11s 972us/step - loss: 0.7575 - accuracy: 0.6891 - 

10885/10885 [==============================] - 11s 995us/step - loss: 0.0828 - accuracy: 0.9699 - auc_3: 0.9575 - precision_3: 0.8513 - recall_3: 0.8128 - val_loss: 2.2849 - val_accuracy: 0.5384 - val_auc_3: 0.9578 - val_precision_3: 0.8520 - val_recall_3: 0.8141
Epoch 31/150
10885/10885 [==============================] - 11s 986us/step - loss: 0.0865 - accuracy: 0.9707 - auc_3: 0.9581 - precision_3: 0.8530 - recall_3: 0.8155 - val_loss: 2.3870 - val_accuracy: 0.5999 - val_auc_3: 0.9584 - val_precision_3: 0.8539 - val_recall_3: 0.8169
Epoch 32/150
10885/10885 [==============================] - 11s 989us/step - loss: 0.0777 - accuracy: 0.9734 - auc_3: 0.9587 - precision_3: 0.8549 - recall_3: 0.8184 - val_loss: 2.5423 - val_accuracy: 0.5549 - val_auc_3: 0.9589 - val_precision_3: 0.8557 - val_recall_3: 0.8197
Epoch 33/150
10885/10885 [==============================] - 11s 986us/step - loss: 0.0842 - accuracy: 0.9713 - auc_3: 0.9591 - precision_3: 0.8565 - recall_3: 0.8209 - val_loss: 2.39

Epoch 60/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0463 - accuracy: 0.9846 - auc_3: 0.9662 - precision_3: 0.8831 - recall_3: 0.8613 - val_loss: 2.5463 - val_accuracy: 0.5521 - val_auc_3: 0.9663 - val_precision_3: 0.8833 - val_recall_3: 0.8616
Epoch 61/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0526 - accuracy: 0.9803 - auc_3: 0.9664 - precision_3: 0.8836 - recall_3: 0.8620 - val_loss: 2.7144 - val_accuracy: 0.5967 - val_auc_3: 0.9664 - val_precision_3: 0.8839 - val_recall_3: 0.8624
Epoch 62/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0518 - accuracy: 0.9813 - auc_3: 0.9665 - precision_3: 0.8842 - recall_3: 0.8629 - val_loss: 2.7070 - val_accuracy: 0.5935 - val_auc_3: 0.9666 - val_precision_3: 0.8845 - val_recall_3: 0.8633
Epoch 63/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0455 - accuracy: 0.9845 - auc_3: 0.9666 - precision_3: 0.8848 - recall_3: 0.8637 - val_loss:

Epoch 90/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0402 - accuracy: 0.9862 - auc_3: 0.9687 - precision_3: 0.8960 - recall_3: 0.8802 - val_loss: 3.3424 - val_accuracy: 0.6008 - val_auc_3: 0.9687 - val_precision_3: 0.8961 - val_recall_3: 0.8805
Epoch 91/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0408 - accuracy: 0.9876 - auc_3: 0.9687 - precision_3: 0.8963 - recall_3: 0.8807 - val_loss: 2.5855 - val_accuracy: 0.5907 - val_auc_3: 0.9687 - val_precision_3: 0.8964 - val_recall_3: 0.8809
Epoch 92/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0328 - accuracy: 0.9895 - auc_3: 0.9688 - precision_3: 0.8966 - recall_3: 0.8811 - val_loss: 2.6277 - val_accuracy: 0.5604 - val_auc_3: 0.9688 - val_precision_3: 0.8967 - val_recall_3: 0.8813
Epoch 93/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0348 - accuracy: 0.9892 - auc_3: 0.9688 - precision_3: 0.8969 - recall_3: 0.8815 - val_loss:

Epoch 120/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0371 - accuracy: 0.9870 - auc_3: 0.9697 - precision_3: 0.9028 - recall_3: 0.8903 - val_loss: 2.7192 - val_accuracy: 0.5549 - val_auc_3: 0.9697 - val_precision_3: 0.9028 - val_recall_3: 0.8904
Epoch 121/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0311 - accuracy: 0.9898 - auc_3: 0.9697 - precision_3: 0.9029 - recall_3: 0.8905 - val_loss: 3.6168 - val_accuracy: 0.6063 - val_auc_3: 0.9697 - val_precision_3: 0.9030 - val_recall_3: 0.8907
Epoch 122/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0343 - accuracy: 0.9889 - auc_3: 0.9697 - precision_3: 0.9031 - recall_3: 0.8908 - val_loss: 3.1954 - val_accuracy: 0.5829 - val_auc_3: 0.9697 - val_precision_3: 0.9032 - val_recall_3: 0.8909
Epoch 123/150
10885/10885 [==============================] - 11s 1ms/step - loss: 0.0373 - accuracy: 0.9892 - auc_3: 0.9697 - precision_3: 0.9032 - recall_3: 0.8910 - val_l

Epoch 150/150
10885/10885 [==============================] - 12s 1ms/step - loss: 0.0272 - accuracy: 0.9919 - auc_3: 0.9701 - precision_3: 0.9073 - recall_3: 0.8969 - val_loss: 2.8887 - val_accuracy: 0.5434 - val_auc_3: 0.9701 - val_precision_3: 0.9073 - val_recall_3: 0.8970


In [50]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 20, 128)           307328    
_________________________________________________________________
batch_normalization_10 (Batc (None, 20, 128)           512       
_________________________________________________________________
activation_10 (Activation)   (None, 20, 128)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 20, 128)           131200    
_________________________________________________________________
batch_normalization_11 (Batc (None, 20, 128)           512       
_________________________________________________________________
activation_11 (Activation)   (None, 20, 128)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)          

In [51]:
# load test data
test_data = pd.read_csv('test_signal.csv')
# Peek at the data
test_data.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X5990,X5991,X5992,X5993,X5994,X5995,X5996,X5997,X5998,X5999
0,C00000,-0.169,-0.174,-0.184,-0.189,-0.200,-0.210,-0.221,-0.226,-0.226,...,0.375,0.386,0.386,0.360,0.323,0.282,0.240,0.184,0.132,0.090
1,C00001,-0.184,-0.174,-0.169,-0.164,-0.158,-0.158,-0.158,-0.153,-0.153,...,-0.086,-0.117,-0.143,-0.148,-0.153,-0.153,-0.153,-0.153,-0.153,-0.158
2,C00002,1.050,1.622,2.143,2.552,2.653,2.675,2.614,2.249,1.734,...,-1.342,-1.354,-1.365,-1.370,-1.376,-1.370,-1.365,-1.354,-1.342,-1.326
3,C00003,2.259,2.667,3.046,3.376,3.583,3.579,3.334,2.872,2.514,...,-0.119,-0.119,-0.119,-0.116,-0.114,-0.109,-0.104,-0.097,-0.090,-0.083
4,C00004,-0.203,-0.203,-0.201,-0.201,-0.201,-0.199,-0.194,-0.189,-0.185,...,-0.008,0.020,0.039,0.044,0.046,0.046,0.049,0.049,0.051,0.051


In [52]:
# Data from the 2nd feature column onwards are input
test_X = test_data.values[:,1:]
test_ids = test_data['ID']
print(test_X.shape)
print(test_ids.shape)

(4000, 6000)
(4000,)


In [53]:
# Deal with missing data

# Replace missing values with Nan
test_X[test_X == ''] = np.nan

# Replace Nan with median
imputer = Imputer(missing_values=np.nan, strategy='mean')
test_X = imputer.fit_transform(test_X)

In [54]:
# # Reduce dimensions of data to make neural networks easier to train, prevent overfitting & remove noise

# pca = PCA(n_components=60) # reduce to 60 dimensions
# pca.fit(test_X)

# print(test_X.shape)
# test_X_pca = pca.transform(test_X)
# print(test_X_pca.shape) 

In [55]:
# Reshape input for Conv1D
test_X_3d = test_X.reshape((4000, 20, 300))
print(test_X_3d.shape)

(4000, 20, 300)


In [56]:
predictions_ontest_softmax = model.predict(test_X_3d)
print(predictions_ontest_softmax)

[[9.5467260e-03 5.1907080e-01 4.6431687e-01 7.0656021e-03]
 [6.5329084e-05 2.6822818e-05 9.9990356e-01 4.2320507e-06]
 [1.0253796e-03 9.9364024e-01 4.5370199e-03 7.9725427e-04]
 ...
 [4.1936369e-06 9.9999201e-01 1.8993610e-07 3.5829360e-06]
 [1.7870280e-03 9.8418242e-01 1.2654434e-02 1.3760749e-03]
 [2.4587891e-04 9.9922824e-01 3.2996910e-04 1.9592635e-04]]


In [57]:
# Decodng output
# Softmax -> Integers
integer_predictions = tf.argmax(input=predictions_ontest_softmax, axis=1)
# Integers -> Class lables
class_predictions = encoder.inverse_transform(integer_predictions)

/usr/local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [58]:
# Merge predictions with IDs
preds = list(zip(test_ids, class_predictions))
print(preds)

[('C00000', 'N'), ('C00001', 'O'), ('C00002', 'N'), ('C00003', 'O'), ('C00004', 'N'), ('C00005', 'O'), ('C00006', 'N'), ('C00007', 'N'), ('C00008', 'N'), ('C00009', 'N'), ('C00010', 'O'), ('C00011', '~'), ('C00012', 'O'), ('C00013', 'O'), ('C00014', 'N'), ('C00015', 'N'), ('C00016', 'N'), ('C00017', 'O'), ('C00018', 'O'), ('C00019', 'N'), ('C00020', 'N'), ('C00021', 'O'), ('C00022', 'N'), ('C00023', 'N'), ('C00024', 'N'), ('C00025', 'N'), ('C00026', 'N'), ('C00027', 'N'), ('C00028', 'N'), ('C00029', 'N'), ('C00030', 'O'), ('C00031', 'O'), ('C00032', 'N'), ('C00033', 'N'), ('C00034', 'O'), ('C00035', 'O'), ('C00036', 'O'), ('C00037', 'O'), ('C00038', 'N'), ('C00039', 'O'), ('C00040', 'O'), ('C00041', 'N'), ('C00042', 'N'), ('C00043', 'N'), ('C00044', 'O'), ('C00045', 'O'), ('C00046', 'N'), ('C00047', 'N'), ('C00048', 'O'), ('C00049', 'N'), ('C00050', 'N'), ('C00051', 'N'), ('C00052', 'O'), ('C00053', 'N'), ('C00054', 'N'), ('C00055', 'O'), ('C00056', 'N'), ('C00057', 'O'), ('C00058', 'N

In [59]:
# As dataframe
predictions = pd.DataFrame(preds, columns=['ID', 'Predicted'])
predictions.head()

,ID,Predicted
0,C00000,N
1,C00001,O
2,C00002,N
3,C00003,O
4,C00004,N


In [60]:
# Ensure Sorted by ID
predictions.sort_values(by=['ID'], inplace=True)
print(predictions)

          ID Predicted
0     C00000         N
1     C00001         O
2     C00002         N
3     C00003         O
4     C00004         N
...      ...       ...
3995  C03995         N
3996  C03996         N
3997  C03997         N
3998  C03998         N
3999  C03999         N

[4000 rows x 2 columns]


In [61]:
# Write to csv file
predictions.to_csv("submission_FCN_without_PCA_with_Dropout_150_epochs.csv", index=False)